In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env')

import os

In [2]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

In [3]:
prompt = """
System: You are a Product Manager, and your job is to design software.
You are provided with a rough description of the software you need to build.
Expand on this description and generate the complete set of functionalities needed to get that software to work.
Don't hesitate to make design choices if the initial description doesn't provide enough information.

Human: {human}

Complete software design:
"""

llm = ChatOpenAI()

product_manager_chain = LLMChain.from_string(
    llm=llm,
    template=prompt,
)

In [4]:
request = """
Build a software for AutoML in Pytorch
The code should be able to search the best architecture and hyperparameters for CNN network based on different metrics.
You can use all the necessary packages you need.
"""

In [20]:
def safe_write(path: str, content: str):
    path = os.path.join("./s12_software/", path)
    
    os.makedirs(os.path.dirname(path), exist_ok=True)
    if os.path.exists(path):
        raise Exception(f"File {path} already exists")
    with open(path, 'w+') as f:
        f.write(content)

## The class structure generation

In [5]:
prompt = """
System: You are a Software Engineer writing code in Python.
Your job is to come up with a detailed description of all the necessary functions, classes, methods, and attributes for the following software description.
Make sure to design a software that incorporates all the best practices of software engineering.
Make sure you describe how all the different classes and functions interact with each other.
The resulting software should be fully functional.

Software description:
{input}

Software design:
"""

tech_lead_chain = LLMChain.from_string(
    llm=llm,
    template=prompt,
)

## File structure

In [7]:
prompt = """
System: You are a Software Engineer and your job is to design software in Python.
Provide a detailed description of the file structure with the required folders and Python files.
Make sure to design a file structure that incorporates all the best practices of software engineering.
Make sure you explain in which folder each file belong to.
Folder and file names should not contain any white spaces.
A human should be able to exactly recreate that file structure.
Make sure that those files account for the design of the software.
Don't generate non-python files.

Software design: {input}

File structure:
"""

file_structure_chain = LLMChain.from_string(
    llm=llm,
    template=prompt,
)



## The file paths

In [10]:
prompt = """
System: Return the complete list of the file paths, including the folder structure, using the following list of python files.
Only return well formed file paths: ./<FOLDER_NAME>/<FILE_NAME>.py

Follow the following template:
<FILE_PATH 1>
<FILE_PATH 2>
...


Human: {input}

File paths list:
"""

file_path_chain = LLMChain.from_string(
    llm=llm,
    template=prompt,
)

## The code

In [15]:
prompt = """
System: You are a Software Engineer and your job is to design software in Python.
Write the code for the following file using the following description. Only return code! The code should be able to run in a python interpreter.
Make sure to implement all the methods and functions.
Make sure to import all the necessary packages.
The code should be complete.

Files structure: {structure}

Software description: {class_structure}

File name: {file}

Python code for this file:
"""

code_chain = LLMChain.from_string(
    llm=llm,
    template=prompt,
)

## Iterating

In [22]:
prompt = """
Return `<TRUE>` if the following Python code contains non-implemented parts and return `<FALSE>` otherwise.

Code: {code}
"""

missing_chain = LLMChain.from_string(
    llm=llm,
    template=prompt,
)


In [23]:
prompt = """
System: You are a Software Engineer. The following Python code may contain non-implemented functions.
If the code contains non-implemented functions, describe what additional functions or classes you would need to implement those missing implementations.
Provide a detailed description of those additional classes or functions that you need to implement.
Make sure to design a software that incorporates all the best practices of software engineering.

Class description: {class_structure}

Code: {code}

Only return text if some functions are not implemented.
The new classes and functions needed:
"""

long_llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k")

new_classes_chain = LLMChain.from_string(
    llm=long_llm,
    template=prompt,
)

## Main app

In [16]:
design = product_manager_chain.run(request)
class_structure = tech_lead_chain.run(design)
file_structure = file_structure_chain.run(class_structure)
files = file_path_chain.run(file_structure)

files_list = files.split('\n')

In [25]:
missing = True
missing_dict = {
    file: True for file in files_list
}

code_dict = {}

while missing:
    missing = False

    new_classes_list = []
    for file in files_list:
        if not missing_dict[file]:
            safe_write(file, code_dict[file])
            continue

        code = code_chain.predict(
            structure=file_structure,
            class_structure=class_structure,
            file=file
        )

        code_dict[file] = code

        response = missing_chain.run(code=code)
        missing_dict[file] = "<TRUE>" in response

        missing = missing or missing_dict[file]

        if missing_dict[file]:
            new_classes = new_classes_chain.run(
                class_structure=class_structure,
                code=code
            )
            new_classes_list.append(new_classes)

    class_structure = "\n\n".join(new_classes_list)

In [12]:
print(files)

./main.py
./user_interface.py
./dataset_selection.py
./search_space_definition.py
./auto_ml_algorithm.py
./model_training_evaluation.py
./results_visualization.py
./model_export_deployment.py
./integration_pytorch.py
./error_handling_logging.py
./customization_extensibility.py
./documentation_support.py
